In [1]:
from pathlib import Path

import numpy as np

INPUT_FILE = Path.cwd() / "day-12.txt"

with INPUT_FILE.open("r") as file:
    input = [line.strip() for line in file.readlines()]

input[0:8]


['abaaacccccccccaaaaaaccccccccccccccccaacccccccccccaacaaaaaaaaaaaaaaaaaccaaaaacccaaaaccccccccccccccccccccccccccccccccccccccccccccccccccccccccaaaaa',
 'abaaacccccccccaaaaaacccccccccccccccaaaaccccccccccaaaaaaaacaaaaaaaaaaaccaaaaaaccaaaacccccccccccccccccccccccccccccccccccccccccccccccccccccccaaaaaa',
 'abaaaccccccccccaaaaacccccccccccccccaaaacccccccccccaaaaacccaaaaaaaaaacccaaaaaacccaaccccccccccccccaaaaacccccccccccccccccccccccccccccccccccccaaaaaa',
 'abccccaaccccccaaaaacccccccccaaaaaccaaaaccccccccccccaaaaacaaaaaaaaacccccaaaaaccccccccccccccccccccaaaaacccccccccccccccccaaaccccaaaccccccccccaaacaa',
 'abcccaaaacccccaaaaacccccccccaaaaacccccccccccccccccaaacaaaaaaaaaacccccccaaaaacccccccccccccccccccaaaaaacccccccccccccccccaaaaccaaaaccccccccccccccaa',
 'abcccaaaaacacccccccccccccccaaaaaaccccccccccccccccccaaccaaaaacaaaaccccccccccccccccccccccccccccccaaaaaaccccccccccccccccccaaaaaaaacccccccccccccccaa',
 'abaaaaaaaaaacccccccccccccccaaaaaaccccccccccccccccccccccaaaacccaaaccccccccccccccccccccccccccccccaaaaaaccc

In [2]:
from pprint import pprint

topo = np.array([list(map(ord, line)) for line in input], dtype=np.int32)
topo -= ord("a")
pprint(topo[20,:])
start_row = (topo == (ord("S") - ord("a"))).nonzero()[0][0]
start_col = (topo == (ord("S") - ord("a"))).nonzero()[1][0]
finish_row = (topo == (ord("E") - ord("a"))).nonzero()[0][0]
finish_col = (topo == (ord("E") - ord("a"))).nonzero()[1][0]
print(f"Start = ({start_col},{start_row}), finish = ({finish_col},{finish_row})")

topo[start_row, start_col] = 0
topo[finish_row, finish_col] = 25


array([-14,   1,   0,   0,   0,   2,   2,   2,   2,   2,   2,   0,   0,
         0,   2,   0,   0,   0,   2,   2,   2,   2,   2,   2,   2,   2,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   0,   0,   0,   0,   0,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   2,   0,   0,   0,   2,
         2,   0,   0,   6,   6,   6,  12,  12,  12,  19,  19,  19,  23,
        23,  23, -28,  25,  25,  25,  25,  24,  24,  24,  21,  21,  15,
        15,  15,  11,  11,  11,   2,   2,   2,   2,   2,   2,   2,   2,
         2], dtype=int32)
Start = (0,20), finish = (119,20)


In [3]:
class Point:
    def __init__(self, x:int, y:int) -> None:
        self.x = x
        self.y = y
        return

    def __eq__(self, other):
        return (self.x == other.x) and (self.y == other.y)

    def __ne__(self, other):
        return not self.__eq__(other)

    def __add__(self, other):
        return Point(self.x+other.x, self.y+other.y)

    def __sub__(self, other):
        return Point(self.x-other.x, self.y-other.y)

    def __neg__(self):
        return Point(-self.x, -self.y)

    def __str__(self) -> str:
        return f"Point(x={self.x},y={self.y})"

NORTH = Point(0, -1)
SOUTH = Point(0, 1)
EAST = Point(1, 0)
WEST = Point(-1, 0)


# Part 1

Shortest path from start to peak.

In [4]:
counts = np.zeros_like(topo)
target = Point(finish_col, finish_row)
paths = [Point(start_col, start_row)]
counts[start_row,start_col] = 1

while True:
    position = paths.pop(0)
    count = counts[position.y, position.x]
    elevation = topo[position.y, position.x]
    for test in [NORTH, SOUTH, EAST, WEST]:
        y_prime = position.y + test.y
        x_prime = position.x + test.x
        if (x_prime < 0) or (y_prime < 0) or (x_prime == topo.shape[1]) or (y_prime == topo.shape[0]):
            continue
        if (counts[y_prime,x_prime] == 0) and ((elevation - topo[y_prime,x_prime]) >= -1):
            counts[y_prime,x_prime] = count + 1
            temp = Point(x_prime, y_prime)
            if temp == target:
                print(f"Part 1: reached target in {count} moves")
                break
            paths.append(temp)
    if temp == target:
        break


Part 1: reached target in 423 moves


# Part 2

Shortest path from any "a" level (sea level) point to the peak.

Brute force solution - check _all_ starting cells at sea level, report shortest.

In [5]:
lengths = []
target = Point(finish_col, finish_row)

for start_y in range(topo.shape[0]):
    for start_x in range(topo.shape[1]):
        if topo[start_y,start_x] == 0:
            counts = np.zeros_like(topo)
            counts[start_y,start_x] = 1
            paths = [Point(start_x,start_y)]
            while len(paths) > 0:
                position = paths.pop(0)
                count = counts[position.y, position.x]
                elevation = topo[position.y, position.x]
                for test in [NORTH, SOUTH, EAST, WEST]:
                    y_prime = position.y + test.y
                    x_prime = position.x + test.x
                    if (x_prime < 0) or (y_prime < 0) or (x_prime == topo.shape[1]) or (y_prime == topo.shape[0]):
                        continue
                    if (counts[y_prime,x_prime] == 0) and ((elevation - topo[y_prime,x_prime]) >= -1):
                        counts[y_prime,x_prime] = count + 1
                        temp = Point(x_prime, y_prime)
                        if temp == target:
                            lengths.append(count)
                            break
                        paths.append(temp)
                if temp == target:
                    break

print(f"Part 2: shortest length is {min(lengths)}")

Part 2: shortest length is 416


# Part 2

Shortest path from any "a" level (sea level) cell to the peak.

Start at peak, report first "a" level cell encountered.

In [16]:
counts = np.zeros_like(topo)
counts[finish_row,finish_col] = 1
paths = [Point(finish_col, finish_row)]

found = False
while len(paths) > 0:
    position = paths.pop(0)
    count = counts[position.y, position.x]
    elevation = topo[position.y, position.x]
    for test in [NORTH, SOUTH, EAST, WEST]:
        y_prime = position.y + test.y
        x_prime = position.x + test.x
        if (x_prime < 0) or (y_prime < 0) or (x_prime == topo.shape[1]) or (y_prime == topo.shape[0]):
            continue
        if (counts[y_prime,x_prime] == 0) and ((topo[y_prime,x_prime] - elevation) >= -1):
            if topo[y_prime,x_prime] == 0:
                print(f"Part 2: reached target in {count} moves")
                found = True
                break
            else:
                counts[y_prime,x_prime] = count + 1
                temp = Point(x_prime, y_prime)
                paths.append(temp)
    if found:
        break


Part 2: reached target in 416 moves
